# WHAT THIS DOES

1. Creates a folder called 'ssd-imagenet'
2. Takes imagenet raw data and converts it into a form for use with the ssd-7 model

In [1]:

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile

from PIL import Image
import pickle

In [23]:
def boundingBoxesSSD(max_folders=50):
    ssd_path = 'ssd-imagenet'
    if not os.path.isdir(ssd_path):
        os.makedirs(ssd_path)
    
    path = 'RealImageNet/'
    boxesPath = os.path.join(path, "LOC_train_solution.csv")
    imageBoxes = [line.rstrip('\n').split(',') for line in open(boxesPath)][1:]
    
    filenames = []
    b1 = []
    b2 = []
    b3 = []
    b4 = []
    labels = []


    for boxes in imageBoxes:
        imageFileName = boxes[0] + ".JPEG"
        boxesSplit = boxes[1].split()
    #     print(boxesSplit)

        for i in range(0, len(boxesSplit), 5):

            box = boxesSplit[i:i+5]
            label = box[0]
            box = box[1:]
            box = [int(b) for b in box]
            filenames.append(imageFileName)
            b1.append(box[0])
            b2.append(box[1])
            b3.append(box[2])
            b4.append(box[3])
            labels.append(label)



    df = pd.DataFrame({'image': filenames, 'xmin': b1, 'xmax': b2, 'ymin': b3, 'ymax': b4, 'class_id': labels})
    
    
    

        
    
    valid_images = []
    traindir = os.path.join(path, 'ImageNetSubsample', 'Data', 'CLS-LOC', 'train')
    
    n = 1
    for folder in os.listdir(traindir):
        if n> max_folders:
            break
        if folder.startswith('n'):

            for f in os.listdir(os.path.join(traindir, folder)):
                if f.endswith('.JPEG'):
                    valid_images.append(f)
                    
        n += 1

    # valid_images
    valid_df = df[df.image.isin(valid_images)]
    classes = valid_df.class_id.unique()
    mapping = dict(zip(classes, range(len(classes))))
    valid_df = valid_df.replace({'class_id': mapping})
    
    with open(os.path.join(ssd_path, 'class_mapping.pickle'), 'wb') as handle:
        pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    df_train, df_val = train_test_split(valid_df, test_size=0.2)
    df_train.to_csv(os.path.join(ssd_path, 'labels_train.csv'), sep=',', index=False)
    df_val.to_csv(os.path.join(ssd_path, 'labels_val.csv'), sep=',', index=False)
    
    
    
#     outpath = '/Users/matt/work/ucsc/class/cmps240/project/ssd_keras/data/imagenet/'
    for img in valid_df.image:
        copyfile(os.path.join(traindir, img.split('_')[0], img), os.path.join(ssd_path, img))
    
    
    size = 224
    for f in os.listdir(ssd_path):
        if f.endswith('.JPEG'):
            img = Image.open(os.path.join(ssd_path, f))
            img = img.resize((size, size), Image.ANTIALIAS)
            img.save(os.path.join(ssd_path, f))
            
    return valid_df

In [24]:
df = boundingBoxesSSD(5)

In [25]:
df

,image,xmin,xmax,ymin,ymax,class_id
1547,n01440764_10048.JPEG,41,142,256,271,0
1548,n01440764_18.JPEG,1,198,317,352,0
1549,n01440764_5781.JPEG,111,182,450,356,0
1550,n01440764_4903.JPEG,201,222,403,325,0
1551,n01440764_11018.JPEG,38,222,258,310,0
1552,n01440764_11830.JPEG,21,233,378,372,0
1553,n01440764_4409.JPEG,3,303,346,486,0
1554,n01440764_11974.JPEG,99,86,466,249,0
1555,n01440764_1313.JPEG,37,106,443,328,0
1556,n01440764_6021.JPEG,151,64,449,223,0
